In [ ]:
# 使用CNN对cifar-10图片数据进行分类
# cifar-10有10类图片，每类6000张，没有重叠

In [1]:
# 载入numpy, time, 自动下载和读取CIFAR-10数据的类
import cifar10_input
import tensorflow as tf
import numpy as np
import time

In [2]:
# 定义batch_size, 训练轮数max_steps, 数据默认路径
max_steps = 3000
batch_size = 128
data_dir = '../models/tutorials/image/cifar10_data/cifar-10-batches-bin'

In [3]:
# 定义初始化权重函数，使用L2 loss
def variable_with_weight_loss(shape, stddev, wl):
    var = tf.Variable(tf.truncated_normal(shape, stddev=stddev))
    if wl is not None:
        weight_loss = tf.multiply(tf.nn.l2_loss(var), wl, name='weight_loss') # 给weight加l2 loss
        tf.add_to_collection('losses', weight_loss)  # 存储loss的collection
    return var

In [4]:
# 生成训练数据和测试数据
# distorted函数中包含了一系列数据增强的操作，如随机水平翻转、随机剪切、设置随机亮度和对比度、标准化等
images_train, labels_train = cifar10_input.distorted_inputs(data_dir=data_dir, batch_size=batch_size)
images_test, labels_test = cifar10_input.inputs(eval_data=True, data_dir=data_dir, batch_size=batch_size)
print('finish generating data')

Filling queue with 20000 CIFAR images before starting to train. This will take a few minutes.
finish generating data


In [5]:
# 创建输入数据的placeholder
# batch_size在后面定义网络结构时被用到，所以不能用None了
image_holder = tf.placeholder(tf.float32, [batch_size, 24, 24, 3])
label_holder = tf.placeholder(tf.int32, [batch_size])

In [6]:
# 第一个卷积层
weight1 = variable_with_weight_loss(shape=[5,5,3,64], stddev=5e-2, wl=0.0) # 卷积核大小5x5,颜色通道3,卷积核数64,没有weight loss
kernel1 = tf.nn.conv2d(image_holder, weight1, [1,1,1,1], padding='SAME')  # 步长为1
bias1 = tf.Variable(tf.constant(0.0, shape=[64]))
conv1 = tf.nn.relu(tf.nn.bias_add(kernel1, bias1))
# 最大池化层尺寸和步长不一致可以增加数据的丰富性
pool1 = tf.nn.max_pool(conv1, ksize=[1,3,3,1], strides=[1,2,2,1], padding='SAME') # 尺寸3x3，步长2x2的最大池化层
# LRN使响应比较大的值变得更大，抑制反馈小的神经元，对ReLU这种没有上限边界的激活函数有用
norm1 = tf.nn.lrn(pool1, 4, bias=1.0, alpha=0.001/9.0, beta=0.75)  # lrn层

In [7]:
# 第二个卷积层，先LRN再最大池化
weight2 = variable_with_weight_loss(shape=[5,5,64,64], stddev=5e-2, wl=0.0)
kernel2 = tf.nn.conv2d(norm1, weight2, [1,1,1,1], padding='SAME')
bias2 = tf.Variable(tf.constant(0.1, shape=[64]))
conv2 = tf.nn.relu(tf.nn.bias_add(kernel2, bias2))
norm2 = tf.nn.lrn(conv2, 4, bias=1.0, alpha=0.001/9.0, beta=0.75)
pool2 = tf.nn.max_pool(norm2, ksize=[1,3,3,1], strides=[1,2,2,1], padding='SAME')

In [8]:
# 全连接层，使用L2正则约束，384个隐含节点
reshape = tf.reshape(pool2, [batch_size, -1])
dim = reshape.get_shape()[1].value
weight3 = variable_with_weight_loss(shape=[dim,384], stddev=0.04, wl=0.004)
bias3 = tf.Variable(tf.constant(0.1, shape=[384]))
local3 = tf.nn.relu(tf.matmul(reshape, weight3) + bias3)

In [9]:
# 全连接层，隐含节点数为前一个的一半，192个
weight4 = variable_with_weight_loss(shape=[384,192], stddev=0.04, wl=0.004)
bias4 = tf.Variable(tf.constant(0.1, shape=[192]))
local4 = tf.nn.relu(tf.matmul(local3, weight4) + bias4)

In [10]:
# 最后一层，模型Inference输出结果。
weight5 = variable_with_weight_loss(shape=[192,10], stddev=1/192.0, wl=0.0)
bias5 = tf.Variable(tf.constant(0.0, shape=[10]))
logits = tf.add(tf.matmul(local4, weight5), bias5)

In [11]:
# 定义loss函数，将cross entropy计算的loss与weight l2 loss相加
def loss(logits, labels):
    labels = tf.cast(labels, tf.int64)
    cross_entropy = tf.nn.sparse_softmax_cross_entropy_with_logits(  # softmax和cross entropy一起计算
        logits=logits, labels=labels, name='cross_entropy_per_example')
    cross_entropy_mean = tf.reduce_mean(cross_entropy, name='cross_entropy')
    tf.add_to_collection('losses', cross_entropy_mean)  # 添加到整体loss
    
    return tf.add_n(tf.get_collection('losses'), name='total_loss') # cross entropy loss和两个全连接层weight l2 loss

In [12]:
# 将logits节点和label_holder传入得到最终loss
loss = loss(logits, label_holder)

In [13]:
# 优化器Adam
train_op = tf.train.AdamOptimizer(1e-3).minimize(loss)

In [14]:
# 计算top-k准确率，这里使用top 1，即输出分数最高的那一类的准确率
top_k_op = tf.nn.in_top_k(logits, label_holder, 1)

In [15]:
# 创建默认session并初始化全部模型参数
sess = tf.InteractiveSession()
tf.global_variables_initializer().run()

In [16]:
# 启动线程队列
tf.train.start_queue_runners()

[<Thread(Thread-4, started daemon 140106178483968)>,
 <Thread(Thread-5, started daemon 140106170091264)>,
 <Thread(Thread-6, started daemon 140106161698560)>,
 <Thread(Thread-7, started daemon 140106153305856)>,
 <Thread(Thread-8, started daemon 140106144913152)>,
 <Thread(Thread-9, started daemon 140105457071872)>,
 <Thread(Thread-10, started daemon 140105448679168)>,
 <Thread(Thread-11, started daemon 140105440286464)>,
 <Thread(Thread-12, started daemon 140105431893760)>,
 <Thread(Thread-13, started daemon 140105423501056)>,
 <Thread(Thread-14, started daemon 140105415108352)>,
 <Thread(Thread-15, started daemon 140105406715648)>,
 <Thread(Thread-16, started daemon 140104853092096)>,
 <Thread(Thread-17, started daemon 140104844699392)>,
 <Thread(Thread-18, started daemon 140104836306688)>,
 <Thread(Thread-19, started daemon 140104827913984)>,
 <Thread(Thread-20, started daemon 140104819521280)>,
 <Thread(Thread-21, started daemon 140104811128576)>,
 <Thread(Thread-22, started daemon

In [17]:
# 训练
for step in range(max_steps):
    start_time = time.time()
    
    image_batch, label_batch = sess.run([images_train, labels_train])
    _, loss_value = sess.run([train_op, loss], feed_dict={image_holder:image_batch, label_holder:label_batch})
    
    duration = time.time() - start_time
    
    if step % 10 == 0:
        examples_per_sec = batch_size / duration
        sec_per_batch = float(duration)
        
        format_str = ('step %d, loss=%.2f (%.1f examples/sec; %.3f sec/batch)')
        print(format_str % (step, loss_value, examples_per_sec, sec_per_batch))

step 0, loss=4.68 (2.9 examples/sec; 43.941 sec/batch)
step 10, loss=3.72 (428.3 examples/sec; 0.299 sec/batch)
step 20, loss=3.09 (444.4 examples/sec; 0.288 sec/batch)
step 30, loss=2.80 (431.8 examples/sec; 0.296 sec/batch)
step 40, loss=2.42 (425.2 examples/sec; 0.301 sec/batch)
step 50, loss=2.17 (425.9 examples/sec; 0.301 sec/batch)
step 60, loss=2.09 (440.0 examples/sec; 0.291 sec/batch)
step 70, loss=2.16 (452.5 examples/sec; 0.283 sec/batch)
step 80, loss=1.99 (440.1 examples/sec; 0.291 sec/batch)
step 90, loss=1.93 (476.1 examples/sec; 0.269 sec/batch)
step 100, loss=1.99 (450.3 examples/sec; 0.284 sec/batch)
step 110, loss=1.81 (437.8 examples/sec; 0.292 sec/batch)
step 120, loss=1.89 (458.6 examples/sec; 0.279 sec/batch)
step 130, loss=1.98 (437.3 examples/sec; 0.293 sec/batch)
step 140, loss=1.84 (465.3 examples/sec; 0.275 sec/batch)
step 150, loss=1.78 (422.2 examples/sec; 0.303 sec/batch)
step 160, loss=1.78 (424.5 examples/sec; 0.301 sec/batch)
step 170, loss=1.75 (462.0

step 1410, loss=1.06 (465.8 examples/sec; 0.275 sec/batch)
step 1420, loss=1.22 (422.7 examples/sec; 0.303 sec/batch)
step 1430, loss=1.21 (401.1 examples/sec; 0.319 sec/batch)
step 1440, loss=1.21 (418.3 examples/sec; 0.306 sec/batch)
step 1450, loss=1.26 (427.7 examples/sec; 0.299 sec/batch)
step 1460, loss=1.18 (453.3 examples/sec; 0.282 sec/batch)
step 1470, loss=0.98 (434.6 examples/sec; 0.295 sec/batch)
step 1480, loss=1.25 (421.8 examples/sec; 0.303 sec/batch)
step 1490, loss=1.24 (444.1 examples/sec; 0.288 sec/batch)
step 1500, loss=1.15 (424.8 examples/sec; 0.301 sec/batch)
step 1510, loss=1.12 (446.5 examples/sec; 0.287 sec/batch)
step 1520, loss=1.23 (442.2 examples/sec; 0.289 sec/batch)
step 1530, loss=1.19 (423.3 examples/sec; 0.302 sec/batch)
step 1540, loss=1.21 (428.4 examples/sec; 0.299 sec/batch)
step 1550, loss=1.07 (432.9 examples/sec; 0.296 sec/batch)
step 1560, loss=1.09 (429.4 examples/sec; 0.298 sec/batch)
step 1570, loss=1.10 (413.2 examples/sec; 0.310 sec/batc

step 2800, loss=1.13 (439.5 examples/sec; 0.291 sec/batch)
step 2810, loss=1.16 (424.5 examples/sec; 0.302 sec/batch)
step 2820, loss=1.00 (438.0 examples/sec; 0.292 sec/batch)
step 2830, loss=1.14 (414.6 examples/sec; 0.309 sec/batch)
step 2840, loss=0.82 (422.5 examples/sec; 0.303 sec/batch)
step 2850, loss=1.10 (412.9 examples/sec; 0.310 sec/batch)
step 2860, loss=0.97 (485.3 examples/sec; 0.264 sec/batch)
step 2870, loss=1.22 (419.6 examples/sec; 0.305 sec/batch)
step 2880, loss=0.95 (453.8 examples/sec; 0.282 sec/batch)
step 2890, loss=0.98 (432.0 examples/sec; 0.296 sec/batch)
step 2900, loss=0.97 (440.7 examples/sec; 0.290 sec/batch)
step 2910, loss=0.95 (462.4 examples/sec; 0.277 sec/batch)
step 2920, loss=0.96 (421.4 examples/sec; 0.304 sec/batch)
step 2930, loss=1.10 (436.6 examples/sec; 0.293 sec/batch)
step 2940, loss=1.32 (434.4 examples/sec; 0.295 sec/batch)
step 2950, loss=1.21 (427.8 examples/sec; 0.299 sec/batch)
step 2960, loss=1.11 (426.9 examples/sec; 0.300 sec/batc

In [18]:
# 测试
num_examples = 10000
import math
num_iter = int(math.ceil(num_examples / batch_size))
true_count = 0
total_sample_count = num_iter * batch_size
step = 0
while step < num_iter:
    image_batch, label_batch = sess.run([images_test, labels_test])
    predictions = sess.run([top_k_op], feed_dict={image_holder:image_batch, label_holder:label_batch})
    true_count += np.sum(predictions)
    step += 1

In [20]:
# 打印结果
precision = true_count / total_sample_count
print('precision @ 1 = %.3f' % precision)

precision @ 1 = 0.715
